In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the data
df = pd.read_csv('recipes.csv', nrows=5000)

In [3]:
df.head(5)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [5]:
df.shape

(5000, 28)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    5000 non-null   int64  
 1   Name                        5000 non-null   object 
 2   AuthorId                    5000 non-null   int64  
 3   AuthorName                  5000 non-null   object 
 4   CookTime                    2270 non-null   object 
 5   PrepTime                    5000 non-null   object 
 6   TotalTime                   5000 non-null   object 
 7   DatePublished               5000 non-null   object 
 8   Description                 4995 non-null   object 
 9   Images                      4999 non-null   object 
 10  RecipeCategory              4999 non-null   object 
 11  Keywords                    4892 non-null   object 
 12  RecipeIngredientQuantities  4999 non-null   object 
 13  RecipeIngredientParts       5000 

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import re

# Custom function to parse 'PT...' format for timedelta
def parse_duration(duration_str):
    try:
        # Match the ISO 8601 duration format
        match = re.match(r'PT(\d+H)?(\d+M)?(\d+S)?', duration_str)
        if match:
            hours = int(match.group(1)[:-1]) if match.group(1) else 0
            minutes = int(match.group(2)[:-1]) if match.group(2) else 0
            seconds = int(match.group(3)[:-1]) if match.group(3) else 0
            return hours * 60 + minutes + seconds / 60
        else:
            return pd.to_timedelta(duration_str).total_seconds() / 60
    except Exception as e:
        # Handle any parsing errors
        print(f"Error parsing {duration_str}: {e}")
        return 0

# Clean and preprocess the data
if 'TotalTime' in df.columns:
    df['TotalTime'] = df['TotalTime'].apply(parse_duration)  # Convert to minutes
else:
    df['TotalTime'] = 0  # Assign 0 if the column is missing

if 'DatePublished' in df.columns:
    df['DatePublished'] = pd.to_datetime(df['DatePublished'])
else:
    df['DatePublished'] = pd.to_datetime('1970-01-01')  # Assign a default date if the column is missing

# Handle missing values
df = df.fillna(0)

# Convert numerical columns to numeric types, coercing errors
numerical_features = ['CookTime', 'PrepTime', 'TotalTime', 'Calories', 'FatContent', 'CarbohydrateContent', 'ProteinContent']
for feature in numerical_features:
    if feature in df.columns:
        df[feature] = pd.to_numeric(df[feature], errors='coerce')

# Encode categorical variables
if 'RecipeCategory' in df.columns:
    # Ensure 'RecipeCategory' is of type str
    df['RecipeCategory'] = df['RecipeCategory'].astype(str)
    le = LabelEncoder()
    df['RecipeCategory'] = le.fit_transform(df['RecipeCategory'])
else:
    df['RecipeCategory'] = 0  # Assign 0 if the column is missing

# Normalize numerical features
existing_numerical_features = [feature for feature in numerical_features if feature in df.columns]

if existing_numerical_features:
    scaler = StandardScaler()
    df[existing_numerical_features] = scaler.fit_transform(df[existing_numerical_features])

# The DataFrame 'df' now includes processed features and is ready for further analysis or modeling


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [15]:
import pandas as pd
import numpy as np

# Assuming df is already loaded and cleaned

# Create a difficulty score based on prep time and number of ingredients
if 'PrepTime' in df.columns and 'RecipeIngredientQuantities' in df.columns:
    # Convert 'RecipeIngredientQuantities' to list if it's a string or similar
    if df['RecipeIngredientQuantities'].dtype == object:
        df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(
            lambda x: len(eval(x)) if isinstance(x, str) and x.startswith('[') else 0
        )
    else:
        # If it's already a numeric type or something else
        df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(lambda x: 1 if pd.notnull(x) else 0)

    df['Difficulty'] = df['PrepTime'] * df['RecipeIngredientQuantities']
else:
    df['Difficulty'] = 0  # Assign a default value if either column is missing

# Create nutritional balance score
if 'ProteinContent' in df.columns and 'CarbohydrateContent' in df.columns and 'FatContent' in df.columns:
    df['NutritionalBalance'] = (df['ProteinContent'] + df['CarbohydrateContent'] + df['FatContent']) / 3
else:
    df['NutritionalBalance'] = 0  # Assign a default value if any column is missing

# Create features for specific health goals
if 'Calories' in df.columns and 'FatContent' in df.columns:
    df['WeightLossFriendly'] = (df['Calories'] < 300) & (df['FatContent'] < 10)
else:
    df['WeightLossFriendly'] = False  # Assign a default value if either column is missing

if 'CarbohydrateContent' in df.columns and 'SugarContent' in df.columns:
    df['DiabetesFriendly'] = (df['CarbohydrateContent'] < 30) & (df['SugarContent'] < 5)
else:
    df['DiabetesFriendly'] = False  # Assign a default value if either column is missing

if 'ProteinContent' in df.columns and 'Calories' in df.columns:
    df['MuscleBuildingFriendly'] = (df['ProteinContent'] > 20) & (df['Calories'] > 300)
else:
    df['MuscleBuildingFriendly'] = False  # Assign a default value if either column is missing

# The DataFrame 'df' now includes new features for difficulty, nutritional balance, and specific health goals


In [17]:
df.head(5)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Difficulty,NutritionalBalance,WeightLossFriendly,DiabetesFriendly,MuscleBuildingFriendly
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,NaN,NaN,0.544022,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,30.2,-0.430428,4.0,0,"c(""Toss 2 cups berries with sugar."", ""Let stan...",NaN,-0.371505,True,False,False
1,39,Biryani,1567,elly9812,NaN,NaN,0.058762,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,20.4,0.853588,6.0,0,"c(""Soak saffron in warm milk for 5 minutes and...",NaN,0.365531,True,False,False
2,40,Best Lemonade,1566,Stephen Little,NaN,NaN,-0.032721,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,77.2,-0.492282,4.0,0,"c(""Into a 1 quart Jar with tight fitting lid, ...",NaN,-0.327385,True,False,False
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,NaN,NaN,0.534078,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,32.1,0.126264,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc...",NaN,-0.052839,True,False,False
4,42,Cabbage Soup,1538,Duckie067,NaN,NaN,-0.026755,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,17.7,-0.406966,4.0,0,"c(""Mix everything together and bring to a boil...",NaN,-0.392302,True,False,False


In [19]:
df.shape

(5000, 33)

In [21]:
def get_user_input():
    location = input("Enter your location: ")
    age = int(input("Enter your age: "))
    budget = float(input("Enter your daily budget: "))
    allergies = input("Enter any allergies (comma-separated): ").split(',')
    diseases = input("Enter any health conditions (comma-separated): ").split(',')
    goal = input("Enter your health goal (Weight loss, Diabetes control, Muscle gain, etc.): ")
    activity_level = input("Enter your activity level (No activity, Less activity, Moderately active, Highly active): ")
    
    return {
        'location': location,
        'age': age,
        'budget': budget,
        'allergies': allergies,
        'diseases': diseases,
        'goal': goal,
        'activity_level': activity_level
    }

In [23]:
def filter_recipes(df, user_input):
    filtered_df = df.copy()
    
    # Filter out recipes with allergens
    for allergen in user_input['allergies']:
        filtered_df = filtered_df[~filtered_df['RecipeIngredientParts'].str.contains(allergen, case=False)]
    
    # Filter based on health conditions and goals
    if 'diabetes' in user_input['diseases']:
        filtered_df = filtered_df[filtered_df['DiabetesFriendly']]
    
    if user_input['goal'] == 'Weight loss':
        filtered_df = filtered_df[filtered_df['WeightLossFriendly']]
    elif user_input['goal'] == 'Muscle gain':
        filtered_df = filtered_df[filtered_df['MuscleBuildingFriendly']]
    
    return filtered_df

def score_recipes(df, user_input):
    df['Score'] = 0
    
    # Score based on nutritional balance
    df['Score'] += df['NutritionalBalance']
    
    # Score based on budget
    df['Score'] += (user_input['budget'] - df['Calories']) / user_input['budget']
    
    # Score based on activity level
    activity_multiplier = {'No activity': 1, 'Less activity': 1.2, 'Moderately active': 1.4, 'Highly active': 1.6}
    df['Score'] *= activity_multiplier[user_input['activity_level']]
    
    return df.sort_values('Score', ascending=False)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def train_recommendation_model(df):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['Description'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

def get_recommendations(df, recipe_index, cosine_sim, num_recommendations=5):
    sim_scores = list(enumerate(cosine_sim[recipe_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]
    recipe_indices = [i[0] for i in sim_scores]
    return df.iloc[recipe_indices]

In [27]:
def generate_diet_chart(df, user_input, cosine_sim, num_days=7, meals_per_day=3):
    filtered_df = filter_recipes(df, user_input)
    scored_df = score_recipes(filtered_df, user_input)
    
    diet_chart = {}
    for day in range(1, num_days + 1):
        diet_chart[f'Day {day}'] = {}
        for meal in range(1, meals_per_day + 1):
            if not scored_df.empty:
                top_recipe = scored_df.iloc[0]
                recommendations = get_recommendations(scored_df, top_recipe.name, cosine_sim)
                diet_chart[f'Day {day}'][f'Meal {meal}'] = {
                    'Recipe': top_recipe['Name'],
                    'Calories': top_recipe['Calories'],
                    'Protein': top_recipe['ProteinContent'],
                    'Carbs': top_recipe['CarbohydrateContent'],
                    'Fat': top_recipe['FatContent'],
                    'Instructions': top_recipe['RecipeInstructions']
                }
                scored_df = scored_df.drop(top_recipe.name)
                scored_df = pd.concat([scored_df, recommendations]).drop_duplicates()
    
    return diet_chart

In [71]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def main():
    # Load the dataset
    df = pd.read_csv('recipes.csv', sep=',', nrows=5000)

    # Create a difficulty score based on prep time and number of ingredients
    if 'PrepTime' in df.columns and 'RecipeIngredientQuantities' in df.columns:
        if df['RecipeIngredientQuantities'].dtype == object:
            df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(
                lambda x: len(eval(x)) if isinstance(x, str) and x.startswith('[') else 0
            )
        else:
            df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(lambda x: 1 if pd.notnull(x) else 0)
    
        df['Difficulty'] = df['PrepTime'] * df['RecipeIngredientQuantities']
    else:
        df['Difficulty'] = 0  # Assign a default value if either column is missing
    
    # Create nutritional balance score
    if 'ProteinContent' in df.columns and 'CarbohydrateContent' in df.columns and 'FatContent' in df.columns:
        df['NutritionalBalance'] = (df['ProteinContent'] + df['CarbohydrateContent'] + df['FatContent']) / 3
    else:
        df['NutritionalBalance'] = 0  # Assign a default value if any column is missing
    
    # Create features for specific health goals
    if 'Calories' in df.columns and 'FatContent' in df.columns:
        df['WeightLossFriendly'] = (df['Calories'] < 300) & (df['FatContent'] < 10)
    else:
        df['WeightLossFriendly'] = False  # Assign a default value if either column is missing
    
    if 'CarbohydrateContent' in df.columns and 'SugarContent' in df.columns:
        df['DiabetesFriendly'] = (df['CarbohydrateContent'] < 30) & (df['SugarContent'] < 5)
    else:
        df['DiabetesFriendly'] = False  # Assign a default value if either column is missing
    
    if 'ProteinContent' in df.columns and 'Calories' in df.columns:
        df['MuscleBuildingFriendly'] = (df['ProteinContent'] > 20) & (df['Calories'] > 300)
    else:
        df['MuscleBuildingFriendly'] = False  # Assign a default value if either column is missing
    
    # Fill NaN values in 'Description' with an empty string
    df['Description'] = df['Description'].fillna('')

    # The DataFrame 'df' now includes new features for difficulty, nutritional balance, and specific health goals

    def get_user_input():
        location = input("Enter your location: ")
        age = int(input("Enter your age: "))
        budget = float(input("Enter your daily budget: "))
        allergies = input("Enter any allergies (comma-separated): ").split(',')
        diseases = input("Enter any health conditions (comma-separated): ").split(',')
        goal = input("Enter your health goal (Weight loss, Diabetes control, Muscle gain, etc.): ")
        activity_level = input("Enter your activity level (No activity, Less activity, Moderately active, Highly active): ")
        
        return {
            'location': location,
            'age': age,
            'budget': budget,
            'allergies': allergies,
            'diseases': diseases,
            'goal': goal,
            'activity_level': activity_level
        }

    def filter_recipes(df, user_input):
        filtered_df = df.copy()
    
        # Filter out recipes with allergens
        for allergen in user_input['allergies']:
            filtered_df = filtered_df[~filtered_df['RecipeIngredientParts'].str.contains(allergen, case=False)]
        
        # Filter based on health conditions and goals
        if 'diabetes' in user_input['diseases']:
            filtered_df = filtered_df[filtered_df['DiabetesFriendly']]
        
        if user_input['goal'] == 'Weight loss':
            filtered_df = filtered_df[filtered_df['WeightLossFriendly']]
        elif user_input['goal'] == 'Muscle gain':
            filtered_df = filtered_df[filtered_df['MuscleBuildingFriendly']]
        
        return filtered_df

    def score_recipes(df, user_input):
        df['Score'] = 0
        
        # Score based on nutritional balance
        df['Score'] += df['NutritionalBalance']
        
        # Score based on budget
        df['Score'] += (user_input['budget'] - df['Calories']) / user_input['budget']
        
        # Score based on activity level
        activity_multiplier = {'No activity': 1, 'Less activity': 1.2, 'Moderately active': 1.4, 'Highly active': 1.6}
        df['Score'] *= activity_multiplier[user_input['activity_level']]
        
        return df.sort_values('Score', ascending=False)
    
    def train_recommendation_model(df):
        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(df['Description'])
        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
        return cosine_sim
    
    def get_recommendations(df, recipe_index, cosine_sim, num_recommendations=5):
        sim_scores = list(enumerate(cosine_sim[recipe_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:num_recommendations+1]
        recipe_indices = [i[0] for i in sim_scores]
        return df.iloc[recipe_indices]
    
    # Train recommendation model
    cosine_sim = train_recommendation_model(df)
    
    # Get user input
    user_input = get_user_input()
    
    # Filter and score recipes
    filtered_recipes = filter_recipes(df, user_input)
    scored_recipes = score_recipes(filtered_recipes, user_input)
    
    # Display top recommendations
    top_recommendations = scored_recipes.head(5)
    print(top_recommendations[['Name', 'Score']])

if __name__ == "__main__":
    main()


Enter your location:  india
Enter your age:  23
Enter your daily budget:  1000
Enter any allergies (comma-separated):  cold
Enter any health conditions (comma-separated):  cold
Enter your health goal (Weight loss, Diabetes control, Muscle gain, etc.):  muscle gain
Enter your activity level (No activity, Less activity, Moderately active, Highly active):  Less activity


                               Name       Score
3201  Indian Tomato Apple Mincement  2055.42676
2741          Hot Chocolate Dry Mix  1950.42156
2564            Sweet Pumpkin Chips  1848.02092
3032      English Christmas Pudding  1070.01468
2326            Light Fruitcake IIi  1031.97116


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
# Import other necessary modules

# Load and preprocess data
@st.cache
def load_data():
    df = pd.read_csv('recipes.csv', sep='\t')
    # Perform data preprocessing steps...
    return df

df = load_data()
cosine_sim = train_recommendation_model(df)

st.title("Personalized Diet Recommendation System")

# Collect user input
location = st.text_input("Enter your location")
age = st.number_input("Enter your age", min_value=1, max_value=120)
budget = st.number_input("Enter your daily budget", min_value=0.0)
allergies = st.multiselect("Select your allergies", options=['Dairy', 'Nuts', 'Gluten', 'Shellfish', 'Eggs'])
diseases = st.multiselect("Select your health conditions", options=['Diabetes', 'Hypertension', 'Heart Disease'])
goal = st.selectbox("Select your health goal", options=['Weight loss', 'Diabetes control', 'Muscle gain', 'General health'])
activity_level = st.select_slider("Select your activity level", options=['No activity', 'Less activity', 'Moderately active', 'Highly active'])

if st.button("Generate Diet Chart"):
    user_input = {
        'location': location,
        'age': age,
        'budget': budget,
        'allergies': allergies,
        'diseases': diseases,
        'goal': goal,
        'activity_level': activity_level
    }
    
    diet_chart = generate_diet_chart(df, user_input, cosine_sim)
    
    # Display diet chart
    for day, meals in diet_chart.items():
        st.subheader(day)
        for meal, details in meals.items():
            st.write(f"**{meal}:**")
            st.write(f"Recipe: {details['Recipe']}")
            st.write(f"Calories: {details['Calories']}")
            st.write(f"Protein: {details['Protein']}g")
            st.write(f"Carbs: {details['Carbs']}g")
            st.write(f"Fat: {details['Fat']}g")